In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error

In [3]:
features_train = pd.read_csv(filepath_or_buffer='data/dengue_features_train.csv')
labels_train = pd.read_csv(filepath_or_buffer='data/dengue_labels_train.csv')
features_test = pd.read_csv(filepath_or_buffer='data/dengue_features_test.csv')

In [4]:
features_train.drop('week_start_date',axis=1,inplace=True)
features_test.drop('week_start_date',axis=1,inplace=True)

In [5]:
labels_train_sj = labels_train[labels_train.city == 'sj'].drop('city',axis=1)
labels_train_iq = labels_train[labels_train.city == 'iq'].drop('city',axis=1)
features_train_sj = features_train[features_train.city == 'sj'].drop('city',axis=1)
features_train_iq = features_train[features_train.city == 'iq'].drop('city',axis=1)
features_test_sj = features_test[features_test.city == 'sj'].drop('city',axis=1)
features_test_iq = features_test[features_test.city == 'iq'].drop('city',axis=1)

In [6]:
features_train_sj = features_train_sj.interpolate()
features_train_iq = features_train_iq.interpolate()
features_test_sj = features_test_sj.interpolate()
features_test_iq = features_test_iq.interpolate()

In [7]:
dataset_sj = features_train_sj.copy()
dataset_sj['total_cases'] = labels_train_sj['total_cases']
dataset_iq = features_train_iq.copy()
dataset_iq['total_cases'] = labels_train_iq['total_cases']

In [12]:
train_target_sj = labels_train_sj['total_cases']

In [13]:
train_sj = dataset_sj.iloc[0:dataset_sj.shape[0]]
test_sj = dataset_sj.iloc[0:dataset_sj.shape[0]]

In [22]:
train_sj = train_sj.drop('total_cases',axis=1)

In [23]:
scorer = make_scorer(mean_absolute_error, False)

In [24]:
model_RFR = RandomForestRegressor(n_estimators=100, random_state=8)
cv_score_RFR = np.sqrt(-cross_val_score(estimator=model_RFR, X=train_sj, y=np.ravel(train_target_sj), cv=10, scoring = scorer)).mean()
cv_score_RFR_std = np.sqrt(-cross_val_score(estimator=model_RFR, X=train_sj, y=np.ravel(train_target_sj), cv=10, scoring = scorer)).std()
print(cv_score_RFR,cv_score_RFR_std)

5.236125627094535 1.4720932506256497


In [25]:
model_RFR.fit(train_sj, np.ravel(train_target_sj))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=8, verbose=0, warm_start=False)

In [26]:
predictions = model_RFR.predict(features_test_sj)

In [20]:
train_sj.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [27]:
predictions = predictions.astype(int)

In [28]:
submission = pd.DataFrame(predictions, columns=["total_cases"])

In [29]:
submission.to_csv('Submission_sj.csv', index = False)

In [30]:
train_target_iq = labels_train_iq['total_cases']

In [31]:
train_iq = dataset_iq.iloc[0:dataset_iq.shape[0]]
test_iq = dataset_iq.iloc[0:dataset_iq.shape[0]]

In [32]:
train_iq = train_iq.drop('total_cases',axis=1)

In [33]:
model_RFR_iq = RandomForestRegressor(n_estimators=100, random_state=8)
cv_score_RFR = np.sqrt(-cross_val_score(estimator=model_RFR_iq, X=train_iq, y=np.ravel(train_target_iq), cv=10, scoring = scorer)).mean()
cv_score_RFR_std = np.sqrt(-cross_val_score(estimator=model_RFR_iq, X=train_iq, y=np.ravel(train_target_iq), cv=10, scoring = scorer)).std()
print(cv_score_RFR,cv_score_RFR_std)

2.751553606621761 0.22011985345892773


In [34]:
model_RFR_iq.fit(train_iq, np.ravel(train_target_iq))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=8, verbose=0, warm_start=False)

In [35]:
predictions_iq = model_RFR_iq.predict(features_test_iq)

In [36]:
predictions_iq = predictions_iq.astype(int)

In [37]:
submission_iq = pd.DataFrame(predictions_iq, columns=["total_cases"])

In [38]:
submission_iq.to_csv('Submission_iq.csv', index = False)